In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

id: a notation for a house
date: Date house was sold
price: Price is prediction target
bedrooms: Number of Bedrooms/House
bathrooms: Number of bathrooms/House
sqft_living: square footage of the home
sqft_lot: square footage of the lot
floors: Total floors (levels) in house
waterfront: House which has a view to a waterfront
view:Has been viewed
condition:How good the condition is ( Overall )
grade: overall grade given to the housing unit, based on King County grading system
sqft_above: square footage of house apart from basement
sqft_basement: square footage of the basement
yr_built: Built Year
yr_renovated: Year when house was renovated
zipcode: zip
lat: Latitude coordinate
long: Longitude coordinate
sqft_living15: Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area
sqft_lot15lot: Size area in 2015(implies-- some renovations)

In [ ]:
df = pd.read_csv("../input/kc_house_data.csv", encoding='ISO-8859-1')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.corr()['price']

In [ ]:
df = df.drop(['id', 'date', 'zipcode'], axis=1)

In [ ]:
df['how_old'] = [(2015-each) for each in df['yr_built']]

In [ ]:
df.corr()['how_old']

In [ ]:
df['log_price'] = [np.log(each) for each in df['price']]

In [ ]:
df.corr()['log_price']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#df['total_sqft'] = df['sqft_living'] + df['sqft_lot'] + df['sqft_above'] + df['sqft_basement']
df = df.drop(['sqft_lot', 'condition', 'long', 'log_price', 'view', 'yr_built', 'yr_renovated'], axis=1)

In [ ]:
df.columns

In [ ]:
df.corr()['price']

In [ ]:
x = np.array(df.drop(['price'], 1))
y = np.array(df['price'])
y = y.reshape(-1, 1)

scaler = MinMaxScaler()
scaler.fit(x)
scaler.fit(y)
x = scaler.transform(x)
y = scaler.transform(y)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=47)

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

classifiers = [SVR(kernel='linear', C=1000, gamma=30), RandomForestRegressor(n_estimators=40, max_depth=None),
               BaggingRegressor(n_estimators=40), LinearRegression(normalize=True, n_jobs=-1)]
names = ['svm', 'rfr', 'br','lr']

for n, c in zip(names, classifiers):
    c.fit(x_train, y_train)
    score = c.score(x_test, y_test)
    print('The accuracy achieved by', n, 'is', score*100)

In [ ]:
rfr = RandomForestRegressor(n_estimators=40, max_depth=None, min_samples_split=2, min_samples_leaf=1)
rfr.fit(x_train, y_train)
acc = rfr.score(x_test, y_test)

print('Accuracy achieved using Random Forest is: ', acc*100)

In [ ]:
import os
from sklearn.tree import export_graphviz

x = 0
for tree_in_forest in rfr.estimators_:
    if(x<1):
        export_graphviz(tree_in_forest)
        os.system('dot -Tpng tree.dot -o tree.png')
    x+=1
    

In [ ]:
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['mean_absolute_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['mean_squared', 'mean_absolute'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(x_test)
y_pred = scaler.inverse_transform(y_pred)
y_pred = rfr.predict(x_test)
score = mean_squared_error(y_test, y_pred)